SYSEN 5888 Spring 2026

Jonathan Lloyd

Homework 2, Question 2


Goal: ConvNets while renowned for their prowess in image processing, have also demonstrated strong capabilities in handling sequential data such as text. In this problem, you will be applying these principles of CNNs to a classic problem in natural language processing - sentiment analysis.

Tools: Numpy, PyTorch, TorchText

Data: IMBD movie reviews dataset torchtext.datasets.IMDB()

Task: Load dataset, use TorchText processing to handle a vocabulary size of 2000 for tokenization & numericalization. Each review in the dataset is already pre-processed and encoded as a sequence of word indexes. A mapping between words and their corresponding indexes is provided using the imdb.get_word_index() method. For consistent input to the model, your task is to pad the reviews or truncate them to a uniform length. This can be achieved using the pad_sequences method from Keras to convert all reviews to a length of 300 words using the maxlen argument in the pad_sequences method.

In [1]:
# update any packages in Colab server
%pip install --upgrade numpy pandas matplotlib torch torchtext tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 79.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 87.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 71.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 81.5 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboa

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext import data
from torchtext.datasets import IMDB
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define fields for text and labels
TEXT = data.Field(lower=True, batch_first=True, include_lengths=False)
LABEL = data.Field(sequential=False)

# Import dataset using splits() — returns (train, test); no path/text_field/label_field needed
train_data, test_data = IMDB.splits(TEXT, LABEL)

# Tokenize and numericalize (build vocab with max_size=2000)
TEXT.build_vocab(train_data, max_size=2000)
LABEL.build_vocab(train_data)

# Pad and truncate: convert examples to index sequences, then pad/truncate to 300
MAX_LEN = 300

def get_sequences_and_labels(dataset):
    """Convert TorchText dataset to lists of word-index sequences and numeric labels using IMDB.get_word_index()."""
    # Get the global word -> index mapping from Keras IMDB
    from torchtext.datasets import IMDB
    word_index = IMDB.get_word_index()
    sequences = []
    labels = []
    for example in dataset:
        # Word indices (use 0 for unknown tokens not in imdb word_index)
        seq = [word_index.get(tok, 0) for tok in example.text]
        sequences.append(seq)
        labels.append(LABEL.vocab.stoi[example.label])
    return sequences, np.array(labels, dtype=np.int64)

train_sequences, train_labels = get_sequences_and_labels(train_data)
test_sequences, test_labels = get_sequences_and_labels(test_data)

# Use Keras pad_sequences: uniform length 300 (pad or truncate)
train_sequences = pad_sequences(train_sequences, maxlen=MAX_LEN, padding='post', truncating='post', value=0)
test_sequences = pad_sequences(test_sequences, maxlen=MAX_LEN, padding='post', truncating='post', value=0)

# Define training and testing datasets (numpy arrays)
# Set aside 1000 training samples for validation; shuffle with fixed seed first
BATCH_SIZE = 32
SHUFFLE_SEED = 42
np.random.seed(SHUFFLE_SEED)
indices = np.random.permutation(len(train_sequences))
train_sequences = train_sequences[indices]
train_labels = train_labels[indices]

n_val = 1000
X_val = train_sequences[:n_val]
y_val = train_labels[:n_val]
X_train = train_sequences[n_val:]
y_train = train_labels[n_val:]

X_test = test_sequences
y_test = test_labels

# Summary
print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}, Test samples: {len(X_test)}")
print(f"Sequence length: {MAX_LEN}, Batch size: {BATCH_SIZE}, Vocab size: {len(TEXT.vocab)}")

OSError: Could not load this library: /usr/local/lib/python3.12/dist-packages/torchtext/lib/libtorchtext.so

Architecture:

The architecture of the convolutional neural network model for this problem is as follows:

Embedding Layer:
Input Vocabulary Size: 2000 words
Embedding Dimension: 16
Input Length: 300 words

Conv1D Layer:
Filters: 128
Kernel Size: 3
Activation: ReLU
Stride: 1
Padding: Valid

GlobalMaxPooling1D Layer

Dense Layer:
Units: 1
Activation: Sigmoid

In [ ]:
# Model Definition


Training: The model should be compiled using the 'binary_crossentropy' as the loss function and 'adam' optimizer. Additionally, 'accuracy' should be assigned as the main metric. A subset of the training data (1000 samples) should be set aside as a validation set, while the rest should be used for training. The model should be trained for a total of 30 (or 10) epochs, with a batch size of 32. After training, the model should be evaluated on the test data to obtain the final accuracy score. This will give a measure of how well the model can generalize to unseen reviews.

In [ ]:
# Train and Run Model

Visualization:
Plot the accuracy and loss for both training and validation datasets across epochs to analyze the performance of the model over epochs.

In [ ]:
# Plots across epochs

Deliverables: 

1. Model Accuracy and Loss Curves: A detailed report of the performance of the model, focusing on accuracy and loss curves.
2. Analysis of Model Performance: A thorough analysis should be conducted to discuss the results obtained from the model. This analysis should include 

a. Whether the model overfits or underfits the training data. 

b. Examination of the loss and accuracy curves to identify potential indicators of the model's behavior (such as plateaus or sharp changes).

3. Code and Resources: Please make sure to submit your working code files along with the final results and the plots.

4. Bonus (+1) Model Optimization: Consider experimenting with other architectures or hyperparameters to further optimize the model's performance. Discuss the outcomes of your experiments and the effect of different parameters on the accuracy and loss